### Загрузка статической модели для русского языка

In [1]:

import ru_core_news_lg
import spacy as sp
from spacy.symbols import ORTH, LEMMA
from IPython.display import Image

nlp = sp.load('ru_core_news_lg')

#### Пример использования

In [2]:
doc = nlp("Я пошла гулять в кино.")
print("doc -- ",doc)
print([(w.text, w.pos_) for w in doc])

doc --  Я пошла гулять в кино.
[('Я', 'PRON'), ('пошла', 'VERB'), ('гулять', 'VERB'), ('в', 'ADP'), ('кино', 'NOUN'), ('.', 'PUNCT')]


##### Глава 2. Токенизация
Самое первое действие любого NLP-приложения — разбор текста на токены, которые могут быть словами, числами или знаками препинания.

In [3]:
doc = nlp(u"Музыка - это искусство, которое дано не всем.")
print([w.text for w in doc])

['Музыка', '-', 'это', 'искусство', ',', 'которое', 'дано', 'не', 'всем', '.']


##### Глава 2. Лемматизация
*Лемма — это базовая, фактически словарная форма токена. Например, лемма токена flying — fly* 

In [4]:
doc = nlp("Яблочный уксус подходит для выпечки тортов.") 
for token in doc:
    print(token.text, " - ", token.lemma_) 

Яблочный  -  яблочный
уксус  -  уксус
подходит  -  подходить
для  -  для
выпечки  -  выпечка
тортов  -  торт
.  -  .


Столбец слева содержит токены, столбец справа — соответствующие леммы.

##### __Глава 2. Использование лемматизации для распознавания смысла__
Нужно сделать так чтобы модель понимала слова использованные в узкиз кругах


In [5]:
doc = nlp("Я хочу поехать в Новочек")
print([w.text for w in doc], "\n")

nlp.get_pipe("attribute_ruler").add([[{"TEXT": "Новочек"}]], {"LEMMA": "Новочеркасск"})
   
print("-->", ['token:%s lemma:%s' % (t.text, t.lemma_) for t in doc], "\n")

# проход по леммам, где Новочек заменился на Новочеркасск
print([w.lemma_ for w in doc])


['Я', 'хочу', 'поехать', 'в', 'Новочек'] 

--> ['token:Я lemma:я', 'token:хочу lemma:хотеть', 'token:поехать lemma:поехать', 'token:в lemma:в', 'token:Новочек lemma:новочек'] 

['я', 'хотеть', 'поехать', 'в', 'новочек']


##### __Глава 2. Частеричная разметка__

In [6]:
doc = nlp("Я хочу уйти спать пораньше.")
print([print(token.text, " - ",token.dep_) for token in doc])

Я  -  nsubj
хочу  -  ROOT
уйти  -  xcomp
спать  -  xcomp
пораньше  -  advmod
.  -  punct
[None, None, None, None, None, None]


In [7]:
doc = nlp("Я прилетела в Питер. Теперь улетаю в Москоу.")
print('%12s %12s %12s' % ("Часть речи", "Токен", "Лемма")) 
print("_" * 40)

for token in doc:
    print('%12s %12s %12s' % (token.tag_, token, token.lemma_)) 

print("_" * 40)
print([w.text for w in doc if w.tag_== 'VERB' or w.tag_== 'PRON'])

print("_" * 40)
print([w.text for w in doc if w.pos_ == 'PROPN'])

  Часть речи        Токен        Лемма
________________________________________
        PRON            Я            я
        VERB    прилетела    прилететь
         ADP            в            в
       PROPN        Питер        питер
       PUNCT            .            .
         ADV       Теперь       теперь
        VERB       улетаю      улетать
         ADP            в            в
       PROPN       Москоу       москоу
       PUNCT            .            .
________________________________________
['Я', 'прилетела', 'улетаю']
________________________________________
['Питер', 'Москоу']


#### Синтаксические отношения. Главный и дочерние элементы.
__Главный__ элемент отношения можно определить с помощью атрибута token.head.

In [8]:
Image(url="images/picture_1.png", width=500, height=300)

In [9]:
doc = nlp("Я прилетела в Питер. Теперь улетаю в Москоу.")
for token in doc:
    print('%12s %12s %12s' % (token.text, token.pos_, token.dep_))
print("_" * 40)
for token in doc:
    print('%12s %12s %12s' % (token.head.text, token.dep_, token.text))

           Я         PRON        nsubj
   прилетела         VERB         ROOT
           в          ADP         case
       Питер        PROPN          obl
           .        PUNCT        punct
      Теперь          ADV       advmod
      улетаю         VERB         ROOT
           в          ADP         case
      Москоу        PROPN          obl
           .        PUNCT        punct
________________________________________
   прилетела        nsubj            Я
   прилетела         ROOT    прилетела
       Питер         case            в
   прилетела          obl        Питер
   прилетела        punct            .
      улетаю       advmod       Теперь
      улетаю         ROOT       улетаю
      Москоу         case            в
      улетаю          obl       Москоу
      улетаю        punct            .


In [10]:
for sent in doc.sents:
    print([w.text for w in sent if w.dep_ == 'ROOT' or w.dep_ == 'obl'])

['прилетела', 'Питер']
['улетаю', 'Москоу']


#### Распознавание именованных сущностей
Именованная сущность (named entity) — объект, на который можно ссылаться по его собственному наименованию.
ent_type - именнованная сущность

LOC - локация

In [11]:
doc = nlp("Я прилетела в Питер. Теперь улетаю в Москоу.")
for token in doc:
    if token.ent_type != 0:
        print('%12s %12s' % (token.text, token.ent_type_))

       Питер          LOC
      Москоу          LOC
